<a href="https://colab.research.google.com/github/webb-e/S2_Landsat_Comparison/blob/main/Lake_wise_cloudiness_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project=" ") # Initialize the library.

# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Load in S2 collection
assetList = ee.data.listAssets('projects/alpod-412314/assets/S2_annual')['assets']
imgs = ee.ImageCollection.fromImages([ee.Image(asset['id']) for asset in assetList])

### define region info
lakeshp = None
region_label = None
def regionfun(region):
  global  lakeshp, region_label
  if region == 'TUK':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/TUK_extraction")
    region_label = 'TUK'
  elif region == 'AND':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/AND_extraction")
    region_label = 'AND'
  elif region == 'MRD':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/MRD_extraction")
    region_label = 'MRD'
  elif region == 'AKCP':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/AKCP_extraction")
    region_label = 'AKCP'
  elif region == 'YKD':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/YKD_extraction")
    region_label = 'YKD'
  elif region == 'YKF':
    lakeshp = ee.FeatureCollection("projects/alpod-412314/assets/Lake_extractions/YKF_extraction")
    region_label = 'YKF'
  else:
    print("Invalid region")



6


In [ ]:
regionfun("AKCP")
def getClouds(image):
##get image year
  S2yr = ee.String(image.get('system:id'))
  year = S2yr.split('_').get(2)
## clip to unet shapefile and create a binary image
  image = image.select('cloud_occurance')
## sum water, masked, and total areas
  reduceroutput = image.reduceRegions(collection =  lakeshp,
                                              scale= 10,
                                              reducer= ee.Reducer.mean())
## add feature properties to  output and convert m2 to km2
  def dealwithoutput(f):
    sample_id = ee.String(f.get('system:index'))
    parsedregion = sample_id.split('_').get(0)
    cloudoccurance = ee.Number(f.get('mean'))
    return f.set({  "year" :  year ,   "cloudiness": cloudoccurance})

  answer = reduceroutput.map(dealwithoutput)
  return answer

S2_intermediate = imgs.map(getClouds)
S2_results = ee.FeatureCollection(S2_intermediate).flatten()

description = 'Lakewise_cloudiness_' + str(region_label)

task = ee.batch.Export.table.toDrive(**{
          'collection': S2_results,
          'selectors': ['year', 'cloudiness','lake_id'],
          'folder': 'Lakewise_csvs',
          'description': description,
          'fileFormat': 'CSV',
    })
task.start()